In [1]:
import pandas as pd
import numpy as np

In [5]:
users_df=pd.read_csv("C:/Users/Gharat/Downloads/recommendation-engine/data/raw/users.csv")

In [6]:
users_df.head()

,User_id,Timestamp,Which of the following best describes you?,Age Group:,Gender Identity,Occupation,Geographical location,Primary Source of Travel Funding:,What are your main reasons for pursuing wellness tourism?,How often do you travel for wellness or mental health purposes?,...,How do you prefer to travel for wellness retreats?,What wellness services are most important to you?,What is your typical budget for a wellness retreat?,Which pricing model do you prefer for wellness retreats,"Would you pay extra for premium wellness experiences (private sessions, personalized coaching, luxury stays)?",What are the biggest barriers preventing you from traveling for wellness more frequently,How important is sustainability and eco-friendliness in your choice of a wellness retreat?,Would you pay extra for eco-friendly and sustainable wellness experiences?,How comfortable are you with using AI-driven recommendations for wellness retreats?,Would you use a mobile app to manage and plan your wellness travel?
0,1,2/8/2025 16:39:08,Traveler - Looking for wellness travel experie...,18 – 24,Female,Student,Urban (metro city),Family-supported,Stress relief and relaxation,Quarterly,...,Solo,"Meditation and mindfulness programs, Professio...","₹25,000 – ₹50,000",All-inclusive package (accommodation + food + ...,Yes,"High costs, Lack of awareness about available ...",2,Yes,1,Yes
1,2,2/8/2025 17:19:51,Traveler - Looking for wellness travel experie...,18 – 24,Female,Student,Urban (metro city),Family-supported,Stress relief and relaxation,Quarterly,...,With friends,Adventure and outdoor activities,"₹50,000 – ₹1,00,000",All-inclusive package (accommodation + food + ...,No,"Time constraints (work, family)",2,No,1,Yes
2,3,2/8/2025 17:20:29,Traveler - Looking for wellness travel experie...,18 – 24,Female,Student,Urban (metro city),Family-supported,Stress relief and relaxation,Rarely,...,Solo,"Meditation and mindfulness programs, Professio...","Under ₹25,000",Pay-per-service (only for selected activities),"Maybe, depending on the price",High costs,2,No,2,Yes
3,4,2/8/2025 19:47:52,Traveler - Looking for wellness travel experie...,18 – 24,Female,Student,Urban (metro city),Self-funded,Stress relief and relaxation,Annually,...,Solo,"Spa and detox treatments, Fitness and yoga cla...","₹50,000 – ₹1,00,000",All-inclusive package (accommodation + food + ...,Yes,"High costs, Lack of awareness about available ...",1,Yes,2,Yes
4,5,2/8/2025 19:50:27,Traveler - Looking for wellness travel experie...,18 – 24,Female,Student,Urban (metro city),Family-supported,Stress relief and relaxation,Annually,...,With family,"Spa and detox treatments, Adventure and outdoo...","Under ₹25,000",Pay-per-service (only for selected activities),Yes,"High costs, Lack of awareness about available ...",2,Yes,1,Yes


In [8]:
# Clean column names: strip whitespaces and convert to snake_case
users_df.columns = users_df.columns.str.strip().str.lower().str.replace(" ", "_")

# Convert 'timestamp' to datetime and extract date features
users_df['timestamp'] = pd.to_datetime(users_df['timestamp'], errors='coerce')
users_df['month'] = users_df['timestamp'].dt.month
users_df['day'] = users_df['timestamp'].dt.day
users_df['hour'] = users_df['timestamp'].dt.hour

In [11]:
users_df.columns = users_df.columns.str.strip()

In [12]:
# Standardize binary categorical columns (yes/no type)
# Binary encoding
binary_cols = [
    'would_you_pay_extra_for_premium_wellness_experiences_(private_sessions,_personalized_coaching,_luxury_stays)?',
    'would_you_pay_extra_for_eco-friendly_and_sustainable_wellness_experiences?',
    'would_you_use_a_mobile_app_to_manage_and_plan_your_wellness_travel?'
]
users_df[binary_cols] = users_df[binary_cols].applymap(lambda x: str(x).strip().lower().replace(" ", "") if isinstance(x, str) else x)
users_df[binary_cols] = users_df[binary_cols].replace({'yes': 1, 'no': 0})

C:\Users\Gharat\AppData\Local\Temp\ipykernel_36196\1656251533.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  users_df[binary_cols] = users_df[binary_cols].applymap(lambda x: str(x).strip().lower().replace(" ", "") if isinstance(x, str) else x)


In [13]:
# Multi-hot encoding
def multi_hot_encode(df, column):
    split_series = df[column].dropna().apply(lambda x: [i.strip().lower() for i in str(x).split(',')])
    unique_options = set(option for sublist in split_series for option in sublist)
    for option in unique_options:
        df[f'{column}_{option.replace(" ", "_").replace("/", "_")}'] = df[column].apply(
            lambda x: 1 if isinstance(x, str) and option in [i.strip().lower() for i in x.split(',')] else 0
        )
    return df

multi_select_cols = [
    'what_are_your_main_reasons_for_pursuing_wellness_tourism?',
    'what_wellness_services_are_most_important_to_you?',
    'what_are_the_biggest_barriers_preventing_you_from_traveling_for_wellness_more_frequently'
]

for col in multi_select_cols:
    users_df = multi_hot_encode(users_df, col)

In [15]:
# One-hot encoding
categorical_cols = [
    'age_group:',
    'gender_identity',
    'primary_source_of_travel_funding:',
    'how_often_do_you_travel_for_wellness_or_mental_health_purposes?',
    'what_type_of_wellness_retreat_location_appeals_to_you_most?',
    'what_is_your_ideal_duration_for_a_wellness_retreat?',
    'how_do_you_prefer_to_travel_for_wellness_retreats?',
    'what_is_your_typical_budget_for_a_wellness_retreat?',
    'which_pricing_model_do_you_prefer_for_wellness_retreats'
]

users_df = pd.get_dummies(users_df, columns=categorical_cols, prefix_sep='_', drop_first=False)

# Drop text columns after encoding
users_df_cleaned = users_df.drop(columns=['timestamp'] + multi_select_cols)

In [16]:
users_df_cleaned.head()

,user_id,which_of_the_following_best_describes_you?,occupation,geographical_location,"would_you_pay_extra_for_premium_wellness_experiences_(private_sessions,_personalized_coaching,_luxury_stays)?",how_important_is_sustainability_and_eco-friendliness_in_your_choice_of_a_wellness_retreat?,would_you_pay_extra_for_eco-friendly_and_sustainable_wellness_experiences?,how_comfortable_are_you_with_using_ai-driven_recommendations_for_wellness_retreats?,would_you_use_a_mobile_app_to_manage_and_plan_your_wellness_travel?,month,...,how_do_you_prefer_to_travel_for_wellness_retreats?_With family,how_do_you_prefer_to_travel_for_wellness_retreats?_With friends,"what_is_your_typical_budget_for_a_wellness_retreat?_Under ₹25,000","what_is_your_typical_budget_for_a_wellness_retreat?_₹1,00,000+","what_is_your_typical_budget_for_a_wellness_retreat?_₹25,000 – ₹50,000","what_is_your_typical_budget_for_a_wellness_retreat?_₹50,000 – ₹1,00,000",which_pricing_model_do_you_prefer_for_wellness_retreats_All-inclusive package (accommodation + food + activities),which_pricing_model_do_you_prefer_for_wellness_retreats_Corporate wellness sponsorship,which_pricing_model_do_you_prefer_for_wellness_retreats_Pay-per-service (only for selected activities),which_pricing_model_do_you_prefer_for_wellness_retreats_Subscription-based (monthly/yearly membership for discounts)
0,1,Traveler - Looking for wellness travel experie...,Student,Urban (metro city),1,2,1,1,1,2,...,False,False,False,False,True,False,True,False,False,False
1,2,Traveler - Looking for wellness travel experie...,Student,Urban (metro city),0,2,0,1,1,2,...,False,True,False,False,False,True,True,False,False,False
2,3,Traveler - Looking for wellness travel experie...,Student,Urban (metro city),"maybe,dependingontheprice",2,0,2,1,2,...,False,False,True,False,False,False,False,False,True,False
3,4,Traveler - Looking for wellness travel experie...,Student,Urban (metro city),1,1,1,2,1,2,...,False,False,False,False,False,True,True,False,False,False
4,5,Traveler - Looking for wellness travel experie...,Student,Urban (metro city),1,2,1,1,1,2,...,True,False,True,False,False,False,False,False,True,False


In [17]:
users_df_cleaned.to_csv("C:/Users/Gharat/Downloads/recommendation-engine/data/processed/users.csv", index=False)